## <font color='limegreen'>HIRING DEMAND FORECASTING</font>
### <font color='limegreen'>I - DATA ANALYSIS</font>

Data Source: Synthetic data created exclusively for this project.

In [1]:
# Python Language Version
from platform import python_version
print('The python version used in this jupyter notebook:', python_version())

The python version used in this jupyter notebook: 3.13.5


In [2]:
# Jupyter execution path
import sys
print(sys.executable)

C:\Users\Alexandre\anaconda3\python.exe


### 1. LOAD PACKAGES

#### Import packages

In [3]:
%%capture output
!pip install -q -r requirements.txt

In [4]:
!pip install pymannkendall

#### Import functions

In [5]:
from utils import *

ModuleNotFoundError: No module named 'torch'

In [ ]:
from utils import normality_shapiro, identify_nulls, analyze_trend, bds_test_acf, evaluate_skewness, breusch_pagan_test, test_ljung_box

In [ ]:
# Record package versions
# !pip install -q -U watermark

In [ ]:
# Packages versions
%reload_ext watermark
%watermark --iversions

### 2. LOAD DATA

In [ ]:
# Load original
df_origin = pd.read_csv('../data/contratacoes_data.csv')

In [ ]:
# Original data copy
df = df_origin.copy()

### 3. UNDERSTANDE AND PROCESS DATA

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
# Dicitionary of variables
dic01 = {
    "data": "Data da contratacao.",
    "total_contratacoes": "Número de empréstimos pessoais efetivados.",}

In [ ]:
# Time series period
print(f"Time series period: {df['data'].min()} to {df['data'].max()}")

In [ ]:
# Convert 'data' variable to datetime and index
df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d')
df = df.set_index('data').asfreq('D')

In [ ]:
# Total null values
df.isnull().sum().sum()

In [ ]:
# Percentage of null values
(df.isnull().mean() * 100).round(2)

### 4. EXPLORATORY ANALYSIS

In [ ]:
# View time series
fig1 = px.line(df, x=df.index, y='contratacoes', title='Série Temporal', 
               width=1000, height=400)
fig1.show()

`Análise Técnica` 

A série apresenta indícios de sazonalidade mensal e tendência crescente, tornando essencial uma análise mais detalhada para identificar padrões recorrentes ao longo do tempo.

In [ ]:
# Distribution of the variable
fig2 = px.histogram(df, x='contratacoes', nbins=30, 
                   title='Distribuição da Variável', 
                   labels={'contratacoes'}, 
                   width=800, height=400)
fig2.show()

`Análise Técnica`

Apesar da presença de valores nulos, a análise da distribuição é fundamental para identificar a assimetria da variável, fornecendo insights valiosos para orientar a escolha do tratamento mais adequado tanto para os valores nulos quanto para possíveis outliers. Nesse contexto, observa-se uma cauda alongada à direita, caracterizando uma assimetria positiva, na qual a média é significativamente maior que a mediana. Essa característica sugere a presença de valores extremos ou outliers que influenciam a média, reforçando a importância de adotar estratégias robustas, como o uso da mediana ou transformações nos dados, para garantir a precisão das análises.

In [ ]:
df.describe()

In [ ]:
# Function to check normality
normality_shapiro(df, 'contratacoes')

`Análise Técnica`
- A saída 'nan' indique que a função detectou que não conseguiu calcular o teste corretamente.

O p-valor é nan geralmente acontece quando há:
- Muitos valores ausentes na variável.
- Poucos dados válidos para o teste estatístico (normalmente precisa de pelo menos 8 pontos de dados).
- Valores constantes (sem variação), o que impossibilita o cálculo das estatísticas.

#### 4.1. CHECK OUTLIERS

In [ ]:
# Boxplot outliers values
fig3 = px.box(df, y='contratacoes', title='Boxplot', 
              labels={'contratacoes'}, width=500, height=400)
fig3.show()

In [ ]:
# Calculate the IQR 
Q1 = df[['contratacoes']].quantile(0.25)
Q3 = df[['contratacoes']].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
# Limits of the variable
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
# Identify outliers 
outliers = (df['contratacoes'] < lower_bound['contratacoes']) | (df['contratacoes'] > upper_bound['contratacoes'])

In [ ]:
# Total outliers
outliers_count = outliers.sum()
print(f"Number of outliers: {outliers_count}")

In [ ]:
# DataFrame with only outliers
df_outliers = df[outliers]

In [ ]:
df_outliers.head()

#### 4.2. NULL VALUE HANDLING

In [ ]:
# Null values
df.isnull().sum()

In [ ]:
# Estatistics numerical variables
df.describe()

In [ ]:
# Function to identifies null values
df_copy, count = identify_nulls(df, 'contratacoes')
print(count)

`Decisão Técnica`

Vamos utlizar a Interpolação Temporal para preencher os nulos isolados e a Média Móvel Ajustada para suavizar os nulos consecutivos, mantendo a tendência e sazonalidade dos dados.

In [ ]:
# Temporal interpolation for isolated nulls
df_copy.loc[df_copy['isolated_null'], 'contratacoes'] = df_copy['contratacoes'].interpolate(method='time')

`Justificativa Técnica`

O uso da **Interpolação Temporal** preserva padrões sazonais, sem criar tendências artificiais e mantém a continuidade da série sem alterar a tendência.

In [ ]:
# Check for nulls after interpolation
print(df_copy.isnull().sum())

In [ ]:
# Adjusted Moving Average for short blocks
df_copy.loc[df_copy['consecutive_null'], 'contratacoes'] = df_copy['contratacoes'].fillna(
    df_copy['contratacoes'].rolling(window=7, min_periods=1, center=True).mean())

`Justificativa Técnica`

O uso da **Média Móvel Ajustada** reduz o impacto de outliers, suaviza variações sem perder a sequência da série e funciona bem para períodos curtos de dados ausentes.

In [ ]:
# Check the nulls after interpolation
print(df_copy.isnull().sum())

In [ ]:
# Delete variables created for identify null values
df_copy.drop(columns=['isolated_null', 'consecutive_null', 'long_null'], inplace=True)

In [ ]:
df_copy.head()

In [ ]:
# Descriptive Statistic
df_copy.describe()

`Análise Técnica`

Os valores estão em uma faixa moderada, variando entre 39 e 491, indicando uma discrepância de escala gerenciável.

### 5. TIME SERIES ANALYSIS - I

#### 5.1. DATA DISTRIBUTION

In [ ]:
# Distribution of the variable 
fig4 = px.histogram(df_copy, x='contratacoes', nbins=30, 
                   title='Distribution of the Variable', 
                   labels={'contratacoes'}, 
                   width=800, height=400)
fig4.show()

In [ ]:
# Functions to check Normality
normality_shapiro(df_copy, 'contratacoes')

#### 5.2. SERIES VIEW

In [ ]:
# View Series
fig5 = px.line(df_copy, x=df_copy.index, y='contratacoes', title='TIME SERIES', 
               width=1000, height=400)
fig5.update_layout(xaxis_title='data')
fig5.show()

#### 5.3. FIRST TREND ANALYSIS 

In [ ]:
# Function to check trend (Test Mann-Kendall)
analyze_trend(df_copy['contratacoes'])

#### 5.4. SECOND TREND ANALYSIS

In [ ]:
# Time series Boxplot
fig6 = px.box(df_copy, x=df_copy.index.year, y="contratacoes", 
             title="Boxplot",
             labels={"contratacoes": "contratacoes", "x": "Year"},
             template="plotly_white", width=800, height=400)
fig6.show()

`Análise Técnica`

- Presença de outliers já identificada na série.
- O aumento da mediana ao longo dos anos confirma uma tendência ascendente.
- As caixas possuem tamanhos diferentes indicando uma dispersão da variável em torno da mediana.
- Um modelo considerando a sazonalidade pode funcionar bem para prever esta série.

#### 5.5. CHECK STATIONARITY

**Teste de Dickey-Fuller**

In [ ]:
# Function to check stationarity
test_dickey_fuller(df_copy['contratacoes'])

In [ ]:
# Save clean file
file_path = os.path.join('..', 'data', 'clean_data.csv')
df_copy.to_csv(file_path, index=True)
print(f"File saved successfully at: {file_path}, including the index!")

### 5.6. RESPONDENDO PERGUNTAS DE NEGÓCIO

In [ ]:
# 1. Como as contratações variam ao longo dos meses?
df_copy['month'] = df_copy.index.month
monthly_avg = df_copy.groupby('month')['contratacoes'].mean().reset_index()

fig7 = px.bar(monthly_avg, x='month', y='contratacoes', 
              labels={'month': 'Mês', 'contratacoes': 'Contratações'},
              title='Média mensal de contratações (2020-2025)', 
              color='month', color_discrete_sequence=px.colors.qualitative.Set1, 
              width=800, height=400)
fig7.show()

In [ ]:
# 2. Em que período do mês há maior volume de contratações? 
# Existem padrões intra-mensais que impactam a demanda?
df_copy['day'] = df_copy.index.day
daily_avg = df_copy.groupby('day')['contratacoes'].mean().reset_index()

fig8 = px.line(daily_avg, x='day', y='contratacoes', 
               labels={'day': 'Dia do Mês', 'contratacoes': 'Contratações'},
               title='Média diária de contratações por dia do mês (2020-2025)', markers=True,
               width=900, height=300)
fig8.show()

`Análise Técnica`

O período entre os dias 1º e 10 do mês concentra o maior volume de contratações.

In [ ]:
# 3. Quais são os períodos de menor demanda que representam oportunidades para investimentos em campanhas de marketing?

# Identificar os dias com menor demanda (menores 25%)
lower_demand_days = df_copy[df_copy['contratacoes'] <= df_copy['contratacoes'].quantile(0.25)].copy() # Use .copy() para evitar SettingWithCopyWarning

# Agrupar os dias de baixa demanda por mês
lower_demand_days['month'] = lower_demand_days.index.month
demand_by_month = lower_demand_days.groupby('month')['contratacoes'].count().reset_index()

# Plotar
fig9 = px.bar(demand_by_month, x='month', y='contratacoes',
                 labels={'month': 'Mês', 'contratacoes': 'Quantidade de dias de baixa demanda'},
                 title='Número de dias de baixa demanda por mês (2020-2025)', color='month',
                 color_discrete_sequence=px.colors.qualitative.Set1)
fig9.show()

`Análise Técnica`

Os meses de dezembro, setembro, agosto, novembro e junho apresentam a menor demanda, destacando-se como oportunidades para investimentos em campanhas de marketing.

In [ ]:
# 4. Como as contratações evoluíram ao longo do tempo? A demanda está em crescimento, queda ou estabilidade?

# Média móvel de 30 dias
df_copy['rolling_mean'] = df_copy['contratacoes'].rolling(window=30).mean()

# Gráfico de linhas
fig10 = px.line(df_copy, x=df_copy.index, y=['contratacoes', 'rolling_mean'],
              labels={'value': 'Contratações', 'variable': 'Legenda'},
              title='Tendência Geral de Contratações (2020-2025)')

# Atualizar nomes das linhas
fig10.for_each_trace(lambda t: t.update(name='Média Móvel (30 dias)' if t.name == 'rolling_mean' else 'Contratações'))

# Ajustar tamanho
fig10.update_layout(width=1000, height=400)

fig10.show()

`Análise Técnica`

A demanda apresenta uma sazonalidade mensal e uma  tendência consistente de crescimento ao longo dos anos, indicando um movimento positivo e sustentável.

In [ ]:
# 5. Há mais contratações nos dias de semana ou nos finais de semana?
import plotly.graph_objects as go

# Adicionar coluna com o dia da semana (0 = segunda, 6 = domingo)
df_copy['weekday'] = df_copy.index.dayofweek

# Calcular médias
day_avg = df_copy.loc[df_copy['weekday'] < 5, 'contratacoes'].mean()
weekend_avg = df_copy.loc[df_copy['weekday'] >= 5, 'contratacoes'].mean()

# Criar gráfico com cores nomeadas
fig11 = go.Figure(data=[
    go.Bar(x=['Dias Úteis', 'Finais de Semana'], y=[day_avg, weekend_avg],
           marker_color=['RoyalBlue', 'DarkOrange'])])

# Layout
fig11.update_layout(
    title='Média de Contratações: Dias Úteis vs Finais de Semana',
    yaxis_title='Contratações',
    xaxis_title='Tipo de Dia',
    width=800, height=400)

fig11.show()

`Análise Técnica`

A análise revelou que, de forma consistente, os dias da semana registram uma demanda significativamente maior por contratações em comparação aos finais de semana.

In [ ]:
df_copy.head()

In [ ]:
# Drop created variables that are not useful for the model
df_copy = df_copy.drop(columns=['month', 'day', 'rolling_mean', 'weekday'])

In [ ]:
df_copy.head()

### 6. ANALYSIS FOR LOGARITHMIC TRANSFORMATION
A checagem de necessidade da transformação logarítmica deve ocorrer antes de qualquer modelagem da série, pois decomposições (aditiva ou multiplicativa) e antes de testes de estacionaridade que podem ser afetados por variações não estabilizadas, resultando em conclusões equivocadas sobre tendências e sazonalidade. 

In [ ]:
# Apply Box-Cox to obtain the lambda value
_, lambda_value = boxcox(df_copy['contratacoes'])

# Determine the series behavior based on the lambda value
if abs(lambda_value) < 0.1:  # Multiplicative behavior
    print(f"Ideal lambda: {lambda_value}. The series has MULTIPLICATIVE behavior.")
elif abs(lambda_value - 1) < 0.1:  # Additive behavior
    print(f"Ideal lambda: {lambda_value}. The series has ADDITIVE behavior.")
else:  # Intermediate behavior
    print(f"Ideal lambda: {lambda_value}. The series has INTERMEDIATE behavior.")

In [ ]:
# Function to check log transformation
log_transformation(df_copy, 'contratacoes')

`Análise Técnica`

Como a série é estacionária, não há necessidade de decomposição temporal. Além disso, a transformação logarítmica não se faz necessária. Portanto, podemos prosseguir com a análise da série.

### 7. TIME SERIES ANALYSIS - II

##### 7.1. CHECK LINEARITY

In [ ]:
# Function to check linearity (Test BDS)
result = bds_test_acf(df_copy['contratacoes'].dropna())
print(result)

##### 7.2. VCHECK SYMMETRY

In [ ]:
# Function to check skewness
result = evaluate_skewness(df_copy, column='contratacoes') 
print(f"Skewness: {result['skewness']:.4f}")
print(f"Interpretation: {result['interpretation']}")

##### 7.3. HETEROSCEDASTICITY
O teste de Breusch-Pagan é um teste estatístico usado para verificar a presença de heterocedasticidade em um modelo de regressão. A heterocedasticidade ocorre quando a variância dos erros (resíduos) de um modelo não é constante ao longo dos valores das variáveis independentes, o que viola uma das premissas básicas dos modelos de regressão linear (homocedasticidade).

In [ ]:
# Function to check heteroscedasticity
result = breusch_pagan_test(df_copy, 'contratacoes')
print("P-value:", result['p_value'])
print("Interpretation:", result['interpretation'])

##### 7.4. AUTOCORRELATION

O teste de Ljung-Box verifica se há autocorrelação nos resíduos de um modelo de série temporal, ajudando a identificar se o modelo captura adequadamente as dependências temporais. Caso a hipótese de independência seja rejeitada, sugere-se a utilização de modelos estatísticos ou algoritmos de machine learning para melhorar a previsão.

In [ ]:
# Function to check autocorrelation (Ljung-Box Test)
message, numerical_result = test_ljung_box(df_copy['contratacoes'].dropna())
print(numerical_result)
print(message)

In [ ]:
df_copy.head()

In [ ]:
df_copy.shape

In [ ]:
df_copy.describe()

`Resumo Técnico`

A série temporal univariada abrange o período de 01/01/2020 a 28/02/2025, totalizando 1.886 registros. O conjunto de dados contém as variáveis 'data' (índice datetime) e 'contratacoes'. A série apresenta uma distribuição não normal, com assimetria à direita, padrões não lineares, e a presença de outliers. Apesar dessas características, a série é estacionária, homocedástica, exibindo sazonalidade, uma tendência de crescimento e correlação significativa.

`Decisão Técnica`

Com base nas características identificadas, a modelagem estatística será conduzida utilizando algoritmos cujos pressupostos estejam alinhados aos padrões observados. O objetivo é gerar previsões para os próximos seis meses, considerando a estrutura da série temporal e suas dinâmicas específicas.

#### POTENTIAL MODELS

Com base nas características da série temporal (não linearidade, sazonalidade, homocedasticidade e outliers), os modelos indicados seriam:

Modelagem Estística:
- SARIMA: Indicado pela presença de sazonalidade e tendência, sendo capaz de capturar padrões autorregressivos, médias móveis e componentes sazonais.

Machine Learning/Deep Learning:
- LSTM e/ou GRU: Para capturar dependências temporais e padrões não lineares, especialmente se a série tiver dependências de longo prazo.


### END OF DATA ANALYSIS
Concluímos a análise da série temporal e os dados estão prontos para a Modelagem Estatística e/ou Machine Learning.